In [1]:
# requirement: h2o version 3.30.0.4
# pip install https://h2o-release.s3.amazonaws.com/h2o/rel-zahradnik/4/Python/h2o-3.30.0.4-py2.py3-none-any.whl

In [2]:
# Reloading
%reload_ext autoreload
%autoreload 2

import h2o
from datetime import datetime
import pandas as pd
import numpy as np
from dgpylib import dg_athena
from dgpylib import dg_s3 
import seaborn as sns

import category_encoders as ce
from sklearn.preprocessing import PowerTransformer

import sys
sys.path.insert(0, '../Modelling')
import helper_function2 as hf

##for data
import pandas as pd
print('pandas version:{}'.format(pd.__version__))
import numpy as np
print('numpy version:{}'.format(np.__version__))
import scipy
print('scipy version:{}'.format(scipy.__version__))
import scipy.stats as stats
## for plotting
import seaborn as sns
from matplotlib import pyplot
import matplotlib.pyplot as plt
##for modelling
import sys
import h2o
print('h2o version:{}'.format(h2o.__version__))
from h2o.estimators import H2OXGBoostEstimator
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators import H2OTargetEncoderEstimator
from h2o.estimators import H2ORandomForestEstimator
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.tree import H2OTree
from h2o.tree import H2ONode
from h2o.tree import H2OSplitNode
from h2o.tree import H2OLeafNode
#import xgboost as xgb
import sklearn
print('sklearn version:{}'.format(sklearn.__version__))
from sklearn import datasets
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import r2_score
import time

pandas version:1.3.4
numpy version:1.20.3
scipy version:1.7.1
h2o version:3.36.0.4
sklearn version:0.24.2


In [3]:
conn1 = dg_s3.Connect('dgdatadump/DS&CA/Pricing/02 Projects/2022/Tel_Reg_2.0/processed_data')
# Load ons data
df_raw = conn1.read('Tel_Reg_Final_Table_Combined_072022_1.csv')

In [4]:
df_raw.shape

(783354, 105)

In [5]:
# df = df_raw.copy()

In [6]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,6 hours 43 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.4
H2O_cluster_version_age:,4 months and 17 days !!!
H2O_cluster_name:,H2O_from_python_ubuntu_mzzc2o
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,26.93 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [7]:
df_raw.columns.to_list()

['offerreferencenumber',
 'offereffectivedatetime',
 'offereffectiveweek',
 'offerclientname',
 'offerbusinesseventsourcename',
 'offerpostcode',
 'offermanufacturerendofguaranteedate',
 'offerendofguaranteeflag',
 'diallerinteractionid',
 'sale',
 'offerperiodofcover',
 'offertotalprice',
 'itemcategoryname',
 'itemsupercategorycode',
 'itemgoodscategoryname',
 'manufacturerbrandname',
 'manufacturergroupname',
 'itempurchaseprice',
 'itempurchasedatetime',
 'itemlocationpostcode',
 'personstatus',
 'persongender',
 'persondisabilitycategory',
 'personaddresspostcode',
 'recordstatus',
 'contractstatuscode',
 'gross',
 'rec_status',
 'movementdate',
 'contracteventdatetime',
 'totalprice',
 'periodofcover',
 'pricingversiondescription',
 'prhidfk',
 'perh_id',
 'requested_purchasedate',
 'requested_purchaprice',
 'fee',
 'planlivecount',
 'plansactivepastoneyearcount',
 'plansactivepastfiveyearcount',
 'plansacceptedpastoneyearcount',
 'plansacceptedpastfiveyearcount',
 'planscancelle

## Pre-processing

In [8]:
def date_fix(df, cols):
    df[cols] = df[cols].apply(pd.to_datetime)
    # df[cols] = df[cols].apply(lambda x: x.dt.date)

In [9]:
date_fix(df_raw, ['offereffectivedatetime'])
    
# month-year column
df_raw['offermonthyear'] = df_raw['offereffectivedatetime'].dt.to_period('M')
# df_demand['startmonthyear'] = df_demand['planaccdate'].dt.to_period('M')
# df_demand['contractmonthyear'] = df_demand['movementdate'].dt.to_period('M')

# month name column
df_raw['offereventmonth'] = df_raw['offereffectivedatetime'].dt.month_name()
# df_demand['startmonth'] = df_demand['planaccdate'].dt.month_name()
# df_demand['contractmonth'] = df_demand['movementdate'].dt.month_name()

In [10]:
df_raw['cancelflag']=np.where(df_raw['contractstatuscode']=='C', 1, 0)
df_raw['cancelflag']=np.where(df_raw['sale']==1, df_raw['cancelflag'], np.nan)

In [11]:
sold_df = df_raw[df_raw['sale']==1]

In [12]:
df_raw[df_raw['cancelflag']==1].sale.value_counts()

1    11315
Name: sale, dtype: int64

In [13]:
df_raw[df_raw['cancelflag']==0].sale.value_counts()

1    158170
Name: sale, dtype: int64

In [14]:
df_raw.sale.mean(), sold_df.cancelflag.mean(),  df_raw.cancelflag.isna().sum()

(0.21635812161551482, 0.0667610703012066, 613869)

In [15]:
# leaky features

leaky_feats = [
    'contractenddate',
    'movementdate',
    'contractmovementdate',
    'retentionflag',
    'contractmonthyear',
    'offereventmonth',
    'contractmonth',
    'contract_age'
]

In [16]:
# redundant features

redundant_feats = [
    'offerpostcode',
    'offeracceptedindicator',
    'persongoldencopyid',
    'itemgoldencopyid',
    'itemeventuserid',
    'contractcreateddate',
    'contractcompanycode',
    'contractschemecode',
    'contractnumber',
    'contractstatusname',  
    'contractdimensionid',
    'contracteventdatetime', 
    'itemmodelnumber',
    'itempurchasedatetime',
    'itemlocationpostcode',
    'personaddresspostcode',
    'planno',
    'planaccdate',
    'planstatus',
    'PmhId',
    'PeriodOfCover',
    'ModelId',
    'PerhId',
    'requested_purchasedate',
    'requested_appliancecode',
    'requested_clientcode',
    'requested_clientgroupcode',
    'requested_itemmanufbrandcode',
    'requested_manugteeperiodlabourmonths',
    'requested_manugteeperiodpartsmonths',
    'offer_request_time',
    'persongoldencopyid_dup',
    'itemcategoryname_dup',
    'manufacturerbrandname_dup',
    'itemlocationpostcode_dup',
    'itemmodelnumber_dup',
    'itemmodelnumber_imputed',
    'itemdiamondcopyid_imputed',
    'itemdiamondcopyid',
    'offersourcereceiveddatetime'
]

In [17]:
columns_to_drop = leaky_feats + redundant_feats
columns_to_drop

['contractenddate',
 'movementdate',
 'contractmovementdate',
 'retentionflag',
 'contractmonthyear',
 'offereventmonth',
 'contractmonth',
 'contract_age',
 'offerpostcode',
 'offeracceptedindicator',
 'persongoldencopyid',
 'itemgoldencopyid',
 'itemeventuserid',
 'contractcreateddate',
 'contractcompanycode',
 'contractschemecode',
 'contractnumber',
 'contractstatusname',
 'contractdimensionid',
 'contracteventdatetime',
 'itemmodelnumber',
 'itempurchasedatetime',
 'itemlocationpostcode',
 'personaddresspostcode',
 'planno',
 'planaccdate',
 'planstatus',
 'PmhId',
 'PeriodOfCover',
 'ModelId',
 'PerhId',
 'requested_purchasedate',
 'requested_appliancecode',
 'requested_clientcode',
 'requested_clientgroupcode',
 'requested_itemmanufbrandcode',
 'requested_manugteeperiodlabourmonths',
 'requested_manugteeperiodpartsmonths',
 'offer_request_time',
 'persongoldencopyid_dup',
 'itemcategoryname_dup',
 'manufacturerbrandname_dup',
 'itemlocationpostcode_dup',
 'itemmodelnumber_dup',


In [18]:
df = hf.drop_columns(df_raw, columns_to_drop)

In [19]:
categorical_cols, integer_cols, float_cols, date_cols, unidentified_cols = hf.cat_num_split(df)

In [20]:
date_cols

['offereffectivedatetime',
 'offermanufacturerendofguaranteedate',
 'datemonth',
 'offe_date_month']

## Pre-processing

## GBM - Feature Selection

In [21]:
# predictors = list(set(categorical_cols + float_cols) - set(['sale']) - set(date_cols))
predictors = ['item_age', 'claimsoneyearpast_flag','purchaseprice_premium_ratio','item_price_bin']
response = 'sale'

In [22]:
df_demand = df.copy()

In [23]:
# convert columns to factors
for i in df.columns[0:]:
    df_demand[i] = df[i].astype('category')
    
#train test split
x_train, x_test, y_train, y_test = train_test_split(df_demand[predictors], df_demand[response], test_size = 0.2, random_state = 1)
train = pd.concat([x_train, y_train], axis=1)
test = pd.concat([x_test, y_test], axis=1)

In [24]:
# convert to h2o dataframe format
hf_train = h2o.H2OFrame(train)
hf_test = h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [25]:
# format target to fit the model
hf_train['sale']=hf_train['sale'].asfactor()
hf_test['sale']=hf_test['sale'].asfactor()

In [26]:
# initialise the estimator 
df_gbm = H2OGradientBoostingEstimator()

In [27]:
start = time.time()
# train the model
df_gbm.train(x = predictors, y = response, training_frame = hf_train)
end = time.time()
print(end - start)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
5.663942813873291


In [28]:
df_gbm.model_performance(test_data=hf_test)


ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.15526134544959025
RMSE: 0.39403216296336807
LogLoss: 0.48577187983330306
Mean Per-Class Error: 0.3781935298114293
AUC: 0.6549397034851501
AUCPR: 0.3860180429969306
Gini: 0.30987940697030014

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.21435855208278745: 


,,0,1,Error,Rate
0,0,101982.0,20977.0,0.1706,(20977.0/122959.0)
1,1,19748.0,13964.0,0.5858,(19748.0/33712.0)
2,Total,121730.0,34941.0,0.2599,(40725.0/156671.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.214359,0.406799,252.0
1,max f2,0.122675,0.578626,383.0
2,max f0point5,0.358459,0.433728,173.0
3,max accuracy,0.494124,0.792336,91.0
4,max precision,0.690054,0.728625,13.0
5,max recall,0.077599,1.000000,398.0
6,max specificity,0.768074,0.999976,0.0
7,max absolute_mcc,0.333229,0.261298,182.0
8,max min_per_class_accuracy,0.176594,0.601981,301.0
9,max mean_per_class_accuracy,0.216516,0.621835,250.0



Gains/Lift Table: Avg response rate: 21.52 %, avg score: 21.66 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010002,0.655139,3.028035,3.028035,0.651563,0.676204,0.651563,0.676204,0.030286,0.030286,202.803467,202.803467,0.025845
1,2,0.020004,0.562817,2.947959,2.987997,0.634333,0.615794,0.642948,0.645999,0.029485,0.059771,194.795931,198.799699,0.050670
2,3,0.030363,0.510213,2.662984,2.877109,0.573013,0.531794,0.619088,0.607034,0.027587,0.087358,166.298381,187.710853,0.072621
3,4,0.040046,0.492129,2.377279,2.756254,0.511536,0.501247,0.593083,0.581456,0.023019,0.110376,137.727946,175.625410,0.089613
4,5,0.050252,0.469985,2.269900,2.657476,0.488430,0.479779,0.571828,0.560805,0.023167,0.133543,126.989970,165.747591,0.106127
5,6,0.100038,0.429025,2.115134,2.387568,0.455128,0.449831,0.513750,0.505577,0.105304,0.238847,111.513381,138.756789,0.176867
6,7,0.150187,0.314683,1.777427,2.183833,0.382461,0.391471,0.469911,0.467475,0.089137,0.327984,77.742719,118.383328,0.226544
7,8,0.200043,0.233606,1.250634,1.951255,0.269108,0.269005,0.419865,0.418011,0.062352,0.390336,25.063383,95.125548,0.242465
8,9,0.300126,0.194465,0.937470,1.613190,0.201722,0.206240,0.347121,0.347392,0.093824,0.484160,-6.253032,61.319033,0.234491
9,10,0.400329,0.180921,0.859960,1.424655,0.185044,0.187032,0.306553,0.307254,0.086171,0.570331,-14.004001,42.465456,0.216612


In [29]:
# classification report
y_true = y_test
y_pred = df_gbm.predict(hf_test)
y_pred = y_pred.as_data_frame()

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [30]:
print(classification_report(y_true, y_pred['predict'], labels=[0,1]))

              precision    recall  f1-score   support

           0       0.84      0.80      0.82    122959
           1       0.38      0.44      0.41     33712

    accuracy                           0.72    156671
   macro avg       0.61      0.62      0.61    156671
weighted avg       0.74      0.72      0.73    156671



In [31]:
# df_xgboost.partial_plot(data = hf_test, cols = ['sale'], plot=True)

In [32]:
# # Retrieve the variable importance
# varimp = df_gbm.varimp(use_pandas=True).sort_values('scaled_importance', ascending=False)
# feats_stored_gbm = list(varimp.head(20)['variable'])
# feats_stored_gbm

In [33]:
# feats_stored_gbm

## GLM - Demand Model

In [34]:
# convert columns to factors
for i in df.columns[0:]:
    df_demand[i] = df[i].astype('category')
    
#train test split
x_train, x_test, y_train, y_test = train_test_split(df_demand[predictors], df_demand[response], test_size = 0.2, random_state = 1)
train_glm = pd.concat([x_train, y_train], axis=1)
test_glm = pd.concat([x_test, y_test], axis=1)

In [35]:
# preserve training & test data's index to be used as a key to merge prediction probability with main dataset 
xtrainglm_index = x_train.index
xtestglm_index = x_test.index

In [36]:
# convert to h2o dataframe format
hf_train_glm = h2o.H2OFrame(train_glm)
hf_test_glm = h2o.H2OFrame(test_glm)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [37]:
# initialise the estimator 
df_glm = H2OGeneralizedLinearEstimator(family= "binomial",
                                          lambda_ = 0,
                                          compute_p_values = True,
                                          remove_collinear_columns = True)


In [38]:
start = time.time()
# train the model
df_glm.train(x = predictors, y = response, training_frame = hf_train_glm)
end = time.time()
print(end - start)

glm Model Build progress: |

/home/ubuntu/anaconda3/lib/python3.9/site-packages/h2o/estimators/estimator_base.py:208: RuntimeWarning: Response is numeric, so the regression model will be trained. However, the cardinality is equaled to two, so if you want to train a classification model, convert the response column to categorical before training.
  warnings.warn(mesg["message"], RuntimeWarning)


██████████████████████████████████████████████████████| (done) 100%
0.44670557975769043


In [39]:
df_glm.model_performance(test_data=hf_test_glm)


ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.1564271516725116
RMSE: 0.3955087251534555
LogLoss: 0.4888610586700356
Null degrees of freedom: 156670
Residual degrees of freedom: 156662
Null deviance: 163170.3340854779
Residual deviance: 153180.70184578633
AIC: 153198.70184578633
AUC: 0.6476592982983632
AUCPR: 0.3780975946241819
Gini: 0.29531859659672643

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.19506790016263878: 


,,0,1,Error,Rate
0,0,98927.0,24032.0,0.1954,(24032.0/122959.0)
1,1,19055.0,14657.0,0.5652,(19055.0/33712.0)
2,Total,117982.0,38689.0,0.275,(43087.0/156671.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.195068,0.404884,296.0
1,max f2,0.146047,0.578361,397.0
2,max f0point5,0.282159,0.430755,231.0
3,max accuracy,0.500448,0.791448,111.0
4,max precision,0.943471,0.750000,0.0
5,max recall,0.144059,1.000000,399.0
6,max specificity,0.943471,0.999984,0.0
7,max absolute_mcc,0.282159,0.259152,231.0
8,max min_per_class_accuracy,0.174391,0.596874,332.0
9,max mean_per_class_accuracy,0.205904,0.620441,283.0



Gains/Lift Table: Avg response rate: 21.52 %, avg score:  0.90 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010002,0.017146,2.188726,2.188726,0.470964,0.020557,0.470964,0.020557,0.021891,0.021891,118.872633,118.872633,0.015149
1,2,0.020004,0.015018,2.360740,2.274733,0.507977,0.016217,0.489470,0.018387,0.023612,0.045503,136.074005,127.473319,0.032491
2,3,0.030006,0.013851,2.262870,2.270779,0.486918,0.014437,0.488619,0.017070,0.022633,0.068136,126.287018,127.077885,0.048585
3,4,0.040001,0.013322,2.006130,2.204648,0.431673,0.013602,0.474390,0.016203,0.020052,0.088188,100.612983,120.464827,0.061399
4,5,0.050003,0.012764,2.031541,2.170022,0.437141,0.012954,0.466939,0.015554,0.020319,0.108507,103.154138,117.002247,0.074545
5,6,0.100006,0.011389,1.904257,2.037140,0.409752,0.012070,0.438346,0.013812,0.095218,0.203726,90.425701,103.713974,0.132157
6,7,0.150002,0.010865,1.216863,1.763737,0.261841,0.011070,0.379516,0.012898,0.060839,0.264565,21.686285,76.373738,0.145972
7,8,0.200005,0.010739,0.718397,1.502394,0.154583,0.010786,0.323281,0.012370,0.035922,0.300486,-28.160273,50.239401,0.128031
8,9,0.300004,0.009994,1.134618,1.379805,0.244144,0.010320,0.296902,0.011687,0.113461,0.413948,13.461806,37.980464,0.145183
9,10,0.400004,0.009623,0.794678,1.233525,0.170996,0.009866,0.265426,0.011231,0.079467,0.493415,-20.532241,23.352521,0.119022


In [40]:
# classification report
y_true = y_test
y_pred = df_glm.predict(hf_test_glm)
y_pred = y_pred.as_data_frame()

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [41]:
print(classification_report(y_true, y_pred['predict'], labels=[0,1]))

              precision    recall  f1-score   support

           0       0.84      0.81      0.83    122959
           1       0.39      0.42      0.40     33712

    accuracy                           0.73    156671
   macro avg       0.61      0.62      0.62    156671
weighted avg       0.74      0.73      0.74    156671



In [42]:
# # saving down the model
# path = 'GLM_demand_model_H2O'
# df_glm.save_mojo(path)


## Demand Model with Cancellation

In [43]:
path = 'GBM_canx_model_H2O'
# load the model
cancellation_gbm =  h2o.import_mojo(path)

generic Model Build progress: |██████████████████████████████████████████████████| (done) 100%
Model Details
H2OGenericEstimator :  Import MOJO Model
Model Key:  Generic_model_python_1660720375071_658


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,31163.0,5.0,5.0,5.0,25.0,32.0,31.12




ModelMetricsBinomialGeneric: generic
** Reported on train data. **

MSE: 0.09482252630532863
RMSE: 0.3079326652132388
LogLoss: 0.3300817422101133
Mean Per-Class Error: 0.21013052220332382
AUC: 0.8271323212274089
AUCPR: 0.7479515038266741
Gini: 0.6542646424548177

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.26119619268152955: 


,,0,1,Error,Rate
0,0,143695.0,4491.0,0.0303,(4491.0/148186.0)
1,1,16040.0,25093.0,0.39,(16040.0/41133.0)
2,Total,159735.0,29584.0,0.1084,(20531.0/189319.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010004,0.983157,4.597746,4.597746,0.998944,0.985610,0.998944,0.985610,0.045997,0.045997,359.774598,359.774598,0.045984
1,2,0.020003,0.979197,4.600175,4.598960,0.999472,0.981313,0.999208,0.983462,0.045997,0.091994,360.017480,359.896007,0.091974
2,3,0.030002,0.973902,4.597743,4.598555,0.998943,0.976648,0.999120,0.981191,0.045973,0.137967,359.774342,359.855459,0.137933
3,4,0.040001,0.965732,4.600175,4.598960,0.999472,0.970428,0.999208,0.978501,0.045997,0.183964,360.017480,359.895959,0.183924
4,5,0.050000,0.948786,4.595312,4.598230,0.998415,0.958148,0.999049,0.974431,0.045949,0.229913,359.531203,359.823015,0.229852
5,6,0.100001,0.713988,4.007953,4.303092,0.870801,0.845971,0.934925,0.910201,0.200399,0.430311,300.795296,330.309156,0.421998
6,7,0.150001,0.317612,3.298064,3.968083,0.716565,0.543861,0.862138,0.788088,0.164904,0.595216,229.806441,296.808251,0.568796
7,8,0.200033,0.164139,1.095257,3.249535,0.237965,0.204146,0.706021,0.642033,0.054798,0.650013,9.525700,224.953476,0.574885
8,9,0.300060,0.129100,0.590121,2.363001,0.128215,0.142801,0.513405,0.475610,0.059028,0.709041,-40.987866,136.300120,0.522506
9,10,0.400224,0.119269,0.505090,1.898021,0.109740,0.123625,0.412380,0.387519,0.050592,0.759633,-49.490991,89.802076,0.459173




Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2022-08-17 08:43:10,0.018 sec,0.0,0.412387,0.523429,0.500000,0.217268,1.000000,0.782732
1,,2022-08-17 08:43:10,0.075 sec,1.0,0.397839,0.491867,0.752117,0.649027,4.584253,0.137165
2,,2022-08-17 08:43:10,0.115 sec,2.0,0.386196,0.470160,0.756584,0.656202,4.597783,0.134498
3,,2022-08-17 08:43:10,0.154 sec,3.0,0.376201,0.452619,0.773160,0.677038,4.598488,0.127800
4,,2022-08-17 08:43:10,0.195 sec,4.0,0.368681,0.439985,0.785397,0.688514,4.599065,0.131968
5,,2022-08-17 08:43:10,0.237 sec,5.0,0.361955,0.428798,0.790455,0.695927,4.597835,0.130151
6,,2022-08-17 08:43:10,0.281 sec,6.0,0.355512,0.418120,0.792510,0.701785,4.597016,0.122043
7,,2022-08-17 08:43:10,0.328 sec,7.0,0.350097,0.409046,0.794632,0.707596,4.597085,0.117590
8,,2022-08-17 08:43:10,0.380 sec,8.0,0.345598,0.401560,0.794943,0.707738,4.597047,0.119296
9,,2022-08-17 08:43:10,0.438 sec,9.0,0.342266,0.395852,0.795014,0.707891,4.597011,0.116671



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,gross,50795.933594,1.000000,0.689579
1,offertotalprice,19040.953125,0.374852,0.258490
2,manufacturerbrandname,2454.453125,0.048320,0.033320
3,offermonthyear,973.787903,0.019171,0.013220
4,item_age,209.066132,0.004116,0.002838
5,item_price_bin,61.770935,0.001216,0.000839
6,plansacceptedpastoneyearcount,59.629097,0.001174,0.000809
7,planlivecount,23.438517,0.000461,0.000318
8,plansacceptedpastfiveyearcount,21.749136,0.000428,0.000295
9,claimpastfiveyearcount_bin,11.908638,0.000234,0.000162


In [44]:
df_withcanx = df.copy()

In [45]:
df_withcanx = df_withcanx[df_withcanx['sale']==1]

In [46]:
# convert columns to factors
for i in df_withcanx.columns[0:]:
    df_withcanx[i] = df_withcanx[i].astype('category')

In [47]:
predictors = ['offertotalprice',
 'offermonthyear',
 'item_age',
 'item_price_bin',
 'plansacceptedpastoneyearcount',
 'planlivecount',
 'plansacceptedpastfiveyearcount',
 'claimamountthreeyeartotal',
 'gross',
 'manufacturerbrandname',
 'claimpastfiveyearcount_bin']

In [48]:
# data type error fix 
ord_encoder = ce.ordinal.OrdinalEncoder( 
                                handle_missing='return_nan',
                                handle_unknown='return_nan')

ord_encoder.fit(df_withcanx['offermonthyear'])
df_withcanx['offermonthyear'] = ord_encoder.transform(df_withcanx['offermonthyear'])

In [49]:
df_withcanx['offermonthyear'].head()

0    1.0
1    2.0
2    3.0
3    3.0
4    4.0
Name: offermonthyear, dtype: float64

In [50]:
response ='cancelflag'

In [51]:
#train test split
x_train, x_test, y_train, y_test = train_test_split(df_withcanx[predictors], df_withcanx[response], test_size = 0.2, random_state = 1)
train = pd.concat([x_train, y_train], axis=1)
test = pd.concat([x_test, y_test], axis=1)

In [52]:
xtrain_index = x_train.index
xtest_index = x_test.index

In [53]:
# convert to h2o dataframe format
hf_train = h2o.H2OFrame(train)
hf_test = h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [54]:
# format target to fit the model
hf_train['cancelflag']=hf_train['cancelflag'].asfactor()
hf_test['cancelflag']=hf_test['cancelflag'].asfactor()

In [55]:
# merge train & test
df_model = hf_train.rbind(hf_test)
# predict train & test sets
predict_te_full = cancellation_gbm.predict(df_model)
#df.describe
#df = pd.DataFrame(df)
#df.shape
df_model = df_model.cbind(predict_te_full[['p1', 'predict']])
df_model = df_model.as_data_frame()

generic prediction progress: |███████████████████████████████████████████████████| (done) 100%


/home/ubuntu/anaconda3/lib/python3.9/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'item_price_bin' has levels not trained on: ["(500, 5000]"]
  warnings.warn(w)
/home/ubuntu/anaconda3/lib/python3.9/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'manufacturerbrandname' has levels not trained on: ["ASUS", "AURA", "BEATSBYDRE", "COOKWORKS", "DIPLOMAT", "ELAC", "FINLUX", "FOCALPOINT", "GAGGIA", "HADEN", ...16 not listed..., "RANGER", "ROBINSONWILLEY", "SEBO", "SKY", "SUBZ", "T4TEC", "XIAOMI", "GLENDIMPLEXBOILERS", "JURA", "NESPRESSO"]
  warnings.warn(w)
/home/ubuntu/anaconda3/lib/python3.9/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'claimpastfiveyearcount_bin' has levels not trained on: ["(100, 300]", "(3, 50]", "(50, 70]", "(70, 100]"]
  warnings.warn(w)


In [56]:
df_model.rename(columns={'p1':'claims_prediction'}, inplace=True)

In [57]:
# combines train & test indices
dfindex = list(xtrain_index) + list(xtest_index)
# reindex claims dataset
df_model = pd.concat([df_model, pd.Series(dfindex)], axis=1)

In [58]:
df_model = df_model.set_index(0)
df_model

,offertotalprice,offermonthyear,item_age,item_price_bin,plansacceptedpastoneyearcount,planlivecount,plansacceptedpastfiveyearcount,claimamountthreeyeartotal,gross,manufacturerbrandname,claimpastfiveyearcount_bin,cancelflag,claims_prediction,predict
0,,,,,,,,,,,,,,
120424,37.20,5,6,"(300, 400]",0,0,0,0.00,37.20,HOOVER,"(-1, 0]",0,0.112626,0
62523,25.44,1,730,"(250, 300]",0,0,0,0.00,25.44,AEG,"(-1, 0]",0,0.083229,0
125759,68.52,6,17,"(500, 5000]",0,0,0,0.00,68.52,ZANUSSI,"(-1, 0]",0,0.112802,0
118633,43.32,1,3,"(500, 5000]",1,2,5,717.56,43.32,WHIRLPOOL,"(3, 50]",0,0.099382,0
120659,43.32,5,0,"(400, 500]",0,0,0,0.00,43.32,BLOMBERG,"(-1, 0]",0,0.105925,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28939,42.60,4,15,"(250, 300]",0,0,0,0.00,42.60,HOTPOINT,"(-1, 0]",0,0.109400,0
100711,63.60,4,181,"(250, 300]",0,0,6,269.00,63.60,INDESIT,"(1, 3]",1,0.157261,0
128751,19.44,1,13,"(250, 300]",0,0,0,0.00,19.44,AEG,"(-1, 0]",0,0.102061,0


In [59]:
df_out = pd.merge(df_raw, df_model[['claims_prediction', 'predict']], how = 'left', left_index = True,   right_index = True)
df_out.head()

,offerreferencenumber,offereffectivedatetime,offereffectiveweek,offerclientname,offerbusinesseventsourcename,offerpostcode,offermanufacturerendofguaranteedate,offerendofguaranteeflag,diallerinteractionid,sale,...,avg_claim_cost_pastoneyr,purchaseprice_premium_ratio,item_price_bin,claimpastfiveyearcount_bin,paymenttype,offermonthyear,offereventmonth,cancelflag,claims_prediction,predict
0,343-003-243-116,2022-02-17,8,WHIRLPOOL,TELEPHONE,SN102SE,2023-02-15,N,NAN,1,...,0.0,6.499833,"(300, 400]","(-1, 0]",DD,2022-02,February,0.0,0.101090,0.0
1,151-007-517-043,2022-03-03,10,HOOVER,TELEPHONE,BL70PN,2022-04-27,N,2002488457S0220303,1,...,0.0,10.745047,"(400, 500]","(-1, 0]",DD,2022-03,March,0.0,0.116939,0.0
2,151-007-779-069,2022-04-06,15,HOOVER,TELEPHONE,IP15HS,2023-04-06,N,100226985480220406,1,...,0.0,8.945151,"(300, 400]","(-1, 0]",DD,2022-04,April,0.0,0.121820,0.0
3,351-015-009-341,2022-04-20,17,D&G,TELEPHONE,LS185QU,2023-03-04,N,1002612646B0220420,1,...,0.0,6.306991,"(200, 250]","(0, 1]",DD,2022-04,April,0.0,0.084026,0.0
4,004-002-137-629,2022-05-20,21,AEG,TELEPHONE,NG138ZD,2022-05-20,N,1002591428J0220520,1,...,0.0,8.650519,"(250, 300]","(-1, 0]",DD,2022-05,May,0.0,0.127471,0.0


In [60]:
df_out['cancelflag'].mean(), df_raw['cancelflag'].mean()

(0.0667610703012066, 0.0667610703012066)

In [61]:
df_out[df_out['sale']==1]['cancelflag'].mean()

0.0667610703012066

In [62]:
df_out.predict.mean()

0.055656842788447354

In [63]:
df_out.cancelflag.isna().sum(), df_out.predict.isna().sum()

(613869, 613869)

In [64]:
df_raw.cancelflag.unique()

array([ 0.,  1., nan])